In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as mplt
import math

from metrics import accuracy_score, RMS_error, crossEntropy_error, confusion_matrix

from LogisticRegression import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K

def loadMNISTData():
	import pickle
	import gzip
	filename = 'mnist.pkl.gz'
	f = gzip.open(filename, 'rb')
	training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
	f.close()
	return training_data, validation_data, test_data

def loadUSPSData():
	from PIL import Image
	import os
	import numpy as np
	USPSMat  = []
	USPSTar  = []
	curPath  = 'USPSdata/Numerals'
	savedImg = []
	for j in range(0,10):
	    curFolderPath = curPath + '/' + str(j)
	    imgs =  os.listdir(curFolderPath)
	    for img in imgs:
	        curImg = curFolderPath + '/' + img
	        if curImg[-3:] == 'png':
	            img = Image.open(curImg,'r')
	            img = img.resize((28, 28))
	            savedImg = img
	            imgdata = (255-np.array(img.getdata()))/255
	            USPSMat.append(imgdata)
	            USPSTar.append(j)
	return np.array(USPSMat), np.array(USPSTar)

MNIST_training, MNIST_validation, MNIST_testing = loadMNISTData()
MNIST_training_X = MNIST_training[0]
MNIST_training_T0 = MNIST_training[1]
MNIST_validation_X = MNIST_validation[0]
MNIST_validation_T0 = MNIST_validation[1]
MNIST_testing_X = MNIST_testing[0]
MNIST_testing_T0 = MNIST_testing[1]
MNIST_training_T = np.array(pd.get_dummies(pd.Series(MNIST_training_T0)))
MNIST_validation_T = np.array(pd.get_dummies(pd.Series(MNIST_validation_T0)))
MNIST_testing_T = np.array(pd.get_dummies(pd.Series(MNIST_testing_T0)))
USPS_X, USPS_T0 = loadUSPSData()
USPS_T = np.array(pd.get_dummies(pd.Series(USPS_T0)))

def plotGraphs(data, plotName):
	fig, ax = mplt.subplots()
	ax.plot(data)
	fig.savefig(plotName+".jpg", dpi=fig.dpi)

print("Logistic Regression:")
lr = 0.005
epochs = 1
batchSize = 100

LogRegressor = LogisticRegression()
LogRegressor.fit( np.transpose(MNIST_training_X), np.transpose(MNIST_training_T), lr, epochs, batchSize)
plotGraphs(LogRegressor.training_error, "LogRegressor_training_error")
plotGraphs(LogRegressor.training_accuracy, "LogRegressor_training_accuracy")
MNIST_training_Err = LogRegressor.training_error[len(LogRegressor.training_error)-1]
MNIST_training_AS = LogRegressor.training_accuracy[len(LogRegressor.training_accuracy)-1]
print("MNIST_training_Err = "+str(MNIST_training_Err))
print("MNIST_training_AS = "+str(MNIST_training_AS))

MNIST_validation_Y =  LogRegressor.predict( np.transpose(MNIST_validation_X) )
MNIST_validation_Y = np.transpose(MNIST_validation_Y)
MNIST_validation_Err = crossEntropy_error( MNIST_validation_T, MNIST_validation_Y )
MNIST_validation_Y = LogRegressor.classificationPredict( np.transpose(MNIST_validation_Y) )
MNIST_validation_Y = np.transpose(MNIST_validation_Y)
MNIST_validation_AS = accuracy_score( MNIST_validation_T, MNIST_validation_Y )
print("MNIST_validation_Err = "+str(MNIST_validation_Err))
print("MNIST_validation_AS = "+str(MNIST_validation_AS))

MNIST_testing_Y = LogRegressor.predict( np.transpose(MNIST_testing_X) )
MNIST_testing_Y = np.transpose(MNIST_testing_Y)
MNIST_testing_Err = crossEntropy_error( MNIST_testing_T, MNIST_testing_Y )
MNIST_testing_Y = LogRegressor.classificationPredict( np.transpose(MNIST_testing_Y) )
MNIST_testing_Y = np.transpose(MNIST_testing_Y)
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T, MNIST_testing_Y )
print("MNIST_testing_Err = "+str(MNIST_testing_Err))
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = LogRegressor.predict( np.transpose(USPS_X) )
USPS_Y = np.transpose(USPS_Y)
USPS_Err = crossEntropy_error( USPS_T, USPS_Y )
USPS_Y = LogRegressor.classificationPredict( np.transpose(USPS_Y) )
USPS_Y = np.transpose(USPS_Y)
USPS_CM, USPS_AS = confusion_matrix( USPS_T, USPS_Y )
print("USPS_Err = "+str(USPS_Err))
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))
MNIST_Y1 = MNIST_testing_Y
USPS_Y1 = USPS_Y
print()
print()


print("SVC:")
kernel = "linear"
gamma = 1

svc = SVC(kernel=kernel, gamma=gamma)
svc.fit( MNIST_training_X[:500], MNIST_training_T0[:500] )

MNIST_training_Y = svc.predict( MNIST_training_X )
MNIST_training_Err = RMS_error( MNIST_training_T0, MNIST_training_Y )
MNIST_training_AS = accuracy_score( MNIST_training_T0, MNIST_training_Y )
print("MNIST_training_Err = "+str(MNIST_training_Err))
print("MNIST_training_AS = "+str(MNIST_training_AS))

MNIST_validation_Y = svc.predict( MNIST_validation_X )
MNIST_validation_Err = RMS_error( MNIST_validation_T0, MNIST_validation_Y )
MNIST_validation_AS = accuracy_score( MNIST_validation_T0, MNIST_validation_Y )
print("MNIST_validation_Err = "+str(MNIST_validation_Err))
print("MNIST_validation_AS = "+str(MNIST_validation_AS))

MNIST_testing_Y = svc.predict( MNIST_testing_X )
MNIST_testing_Err = RMS_error( MNIST_testing_T0, MNIST_testing_Y )
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T0, MNIST_testing_Y )
print("MNIST_testing_Err = "+str(MNIST_testing_Err))
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = svc.predict( USPS_X )
USPS_Err = RMS_error( USPS_T0, USPS_Y )
USPS_CM, USPS_AS = confusion_matrix( USPS_T0, USPS_Y )
print("USPS_Err = "+str(USPS_Err))
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))
MNIST_Y2 = np.array(pd.get_dummies(pd.Series(MNIST_testing_Y))) 
USPS_Y2 = np.array(pd.get_dummies(pd.Series(USPS_Y))) 
print()
print()


print("RF:")
n_estimators = 5
criterion = "gini"

rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion)
rf.fit( MNIST_training_X, MNIST_training_T )

MNIST_training_Y = rf.predict( MNIST_training_X )
MNIST_training_Err = RMS_error( MNIST_training_T, MNIST_training_Y )
MNIST_training_AS = accuracy_score( MNIST_training_T, MNIST_training_Y )
print("MNIST_training_Err = "+str(MNIST_training_Err))
print("MNIST_training_AS = "+str(MNIST_training_AS))

MNIST_validation_Y = rf.predict( MNIST_validation_X )
MNIST_validation_Err = RMS_error( MNIST_validation_T, MNIST_validation_Y )
MNIST_validation_AS = accuracy_score( MNIST_validation_T, MNIST_validation_Y )
print("MNIST_validation_Err = "+str(MNIST_validation_Err))
print("MNIST_validation_AS = "+str(MNIST_validation_AS))

MNIST_testing_Y = rf.predict( MNIST_testing_X )
MNIST_testing_Err = RMS_error( MNIST_testing_T, MNIST_testing_Y )
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T, MNIST_testing_Y )
print("MNIST_testing_Err = "+str(MNIST_testing_Err))
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = rf.predict( USPS_X )
USPS_Err = RMS_error( USPS_T, USPS_Y )
USPS_CM, USPS_AS = confusion_matrix( USPS_T, USPS_Y )
print("USPS_Err = "+str(USPS_Err))
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))
MNIST_Y3 = MNIST_testing_Y
USPS_Y3 = USPS_Y
print()
print()


def recreateImages(concatenatedImgs):
	input_size = len(concatenatedImgs[0])
	op = []
	for concatenatedImg in concatenatedImgs:
		op.append( np.resize(concatenatedImg, ( (int)(math.sqrt(input_size)), (int)(math.sqrt(input_size)), 1 )) )
	return np.array(op)
    
print("NN:")
input_size = 784
output_size = 10

#DNN CODE-
'''
lr = 0.1
decay = 1e-6
momentum= 0.3
first_layer_number_of_nodes = 500
second_layer_number_of_nodes = 500
DNN = Sequential()
DNN.add(Dense(first_layer_number_of_nodes, input_dim=input_size))
DNN.add(Activation("relu"))
DNN.add(Dropout(0.1))
DNN.add(Dense(second_layer_number_of_nodes))
DNN.add(Activation("relu"))
DNN.add(Dropout(0.1))
DNN.add(Dense(output_size))
DNN.add(Activation("softmax"))
opt = optimizers.SGD(lr=lr,decay=decay,momentum=momentum,nesterov=True)
DNN.compile(optimizer=opt, loss="mean_squared_error", metrics=['accuracy'])

numOfEpochs = 25
modelBatchSize = 200 
tbBatchSize = 32
earlyPatience = 100
tb = TensorBoard(log_dir='logs', batch_size=tbBatchSize, write_graph= True)
es = EarlyStopping(monitor='val_loss', verbose=0, patience=earlyPatience, mode='min')
history = DNN.fit( MNIST_training_X, MNIST_training_T, validation_data=(MNIST_validation_X, MNIST_validation_T), epochs=numOfEpochs, batch_size=modelBatchSize, verbose=0, callbacks = [tb, es])

%matplotlib inline
df = pd.DataFrame(history.history)
df.plot(subplots=True, grid=True, figsize=(10,15))

MNIST_training_Err = history.history['loss'][len(history.history['loss'])-1]
MNIST_training_AS = history.history['acc'][len(history.history['acc'])-1] * 100
print("MNIST_training_Err = "+str(MNIST_training_Err))
print("MNIST_training_AS = "+str(MNIST_training_AS))

MNIST_validation_Err = history.history['val_loss'][len(history.history['val_loss'])-1]
MNIST_validation_AS = history.history['acc'][len(history.history['acc'])-1] * 100
print("MNIST_validation_Err = "+str(MNIST_validation_Err))
print("MNIST_validation_AS = "+str(MNIST_validation_AS))

MNIST_testing_Y = DNN.predict( MNIST_testing_X )
MNIST_testing_Err = RMS_error( MNIST_testing_T, MNIST_testing_Y )
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T, MNIST_testing_Y )
print("MNIST_testing_Err = "+str(MNIST_testing_Err))
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = DNN.predict( USPS_X )
USPS_Err = RMS_error( USPS_T, USPS_Y )
USPS_CM, USPS_AS = confusion_matrix( USPS_T, USPS_Y )
print("USPS_Err = "+str(USPS_Err))
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))
MNIST_Y4 = MNIST_testing_Y
USPS_Y4 = USPS_Y
print()
print()

'''
#CNN CODE-
lr = 0.1
decay = 1e-6
momentum= 0.3
inputShape = ((int)(math.sqrt(input_size)), (int)(math.sqrt(input_size)), 1)
chanDim = 1
CNN = Sequential()
# CONV => RELU => POOL layer set
CNN.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
CNN.add(Activation("relu"))
CNN.add(BatchNormalization(axis=chanDim))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Dropout(0.25))
# (CONV => RELU) * 2 => POOL layer set
CNN.add(Conv2D(64, (3, 3), padding="same"))
CNN.add(Activation("relu"))
CNN.add(BatchNormalization(axis=chanDim))
CNN.add(Conv2D(64, (3, 3), padding="same"))
CNN.add(Activation("relu"))
CNN.add(BatchNormalization(axis=chanDim))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Dropout(0.25))
# FC => RELU layers
CNN.add(Flatten())
CNN.add(Dense(512))
CNN.add(Activation("relu"))
CNN.add(BatchNormalization())
CNN.add(Dropout(0.5))
# softmax classifier
CNN.add(Dense(output_size))
CNN.add(Activation("softmax"))
opt = optimizers.SGD( lr=lr, decay=decay, momentum=momentum, nesterov=True)
CNN.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])

numOfEpochs = 1
modelBatchSize = 200
history = CNN.fit( recreateImages(MNIST_training_X), MNIST_training_T, validation_data=(recreateImages(MNIST_validation_X), MNIST_validation_T), epochs=numOfEpochs, batch_size=modelBatchSize)

%matplotlib inline
df = pd.DataFrame(history.history)
df.plot(subplots=True, grid=True, figsize=(10,15))

MNIST_training_Err = history.history['loss'][len(history.history['loss'])-1]
MNIST_training_AS = history.history['acc'][len(history.history['acc'])-1] * 100
print("MNIST_training_Err = "+str(MNIST_training_Err))
print("MNIST_training_AS = "+str(MNIST_training_AS))

MNIST_validation_Err = history.history['val_loss'][len(history.history['val_loss'])-1]
MNIST_validation_AS = history.history['acc'][len(history.history['acc'])-1] * 100
print("MNIST_validation_Err = "+str(MNIST_validation_Err))
print("MNIST_validation_AS = "+str(MNIST_validation_AS))

MNIST_testing_Y = CNN.predict(recreateImages(MNIST_testing_X), batch_size=32)
MNIST_testing_Err = RMS_error( MNIST_testing_T, MNIST_testing_Y )
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T, MNIST_testing_Y )
print("MNIST_testing_Err = "+str(MNIST_testing_Err))
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = CNN.predict(recreateImages(USPS_X), batch_size=32)
USPS_Err = RMS_error( USPS_T, USPS_Y )
USPS_CM, USPS_AS = confusion_matrix( USPS_T, USPS_Y )
print("USPS_Err = "+str(USPS_Err))
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))
MNIST_Y4 = MNIST_testing_Y
USPS_Y4 = USPS_Y
print()
print()


#COMBINING THEM
print("Combined Classification: ")
MNIST_Y = []
for y1, y2, y3, y4 in zip(MNIST_Y1, MNIST_Y2, MNIST_Y3, MNIST_Y4):
    y1Class = np.argmax(y1)
    y2Class = np.argmax(y2)
    y3Class = np.argmax(y3)
    y4Class = np.argmax(y4)
    yClasses = np.array([ y1Class, y2Class, y3Class, y4Class ])
    MNIST_Y.append( (stats.mode(yClasses))[0][0] )
MNIST_Y = np.array(MNIST_Y)
MNIST_testing_CM, MNIST_testing_AS = confusion_matrix( MNIST_testing_T0, MNIST_Y )
print("MNIST_testing_CM = "+str(MNIST_testing_CM))
print("MNIST_testing_AS = "+str(MNIST_testing_AS))

USPS_Y = []
for y1, y2, y3, y4 in zip(USPS_Y1, USPS_Y2, USPS_Y3, USPS_Y4):
    y1Class = np.argmax(y1)
    y2Class = np.argmax(y2)
    y3Class = np.argmax(y3)
    y4Class = np.argmax(y4)
    yClasses = np.array([ y1Class, y2Class, y3Class, y4Class ])
    USPS_Y.append( (stats.mode(yClasses))[0][0] )
USPS_Y = np.array(USPS_Y)
USPS_CM, USPS_AS = confusion_matrix( USPS_T0, USPS_Y )
print("USPS_CM = "+str(USPS_CM))
print("USPS_AS = "+str(USPS_AS))  
    
    